In [2]:
''' A exécuter en premier à chaque fois'''
import pandas as pd
import numpy as np
import random
from path import Path as path
from sklearn.model_selection import LeavePOut, cross_val_score, StratifiedKFold, permutation_test_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier as RF
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import SVC as SVM
from sklearn.tree import DecisionTreeClassifier
from numpy.random import permutation
from scipy.io import savemat, loadmat
# from mlxtend.feature_selection import SequentialFeatureSelector as SFS


def CleanDataset(dataset, columns_to_drop, threshold=30):
    '''Nettoie les données.'''
    dropped_columns = []
    dropped_subjects = []
    subject_list = dataset['CODE']
    for column in columns_to_drop:
        if column in dataset.keys():
            try:
                dataset = dataset.drop(column, 1)
                dropped_columns.append(column)
            except:
                print('there was a problem droping', column)
                None
            del columns_to_drop[columns_to_drop.index(column)]
            
    for thresh in reversed(range(1,threshold+1)):
        dropped_counter = {}
        for column in dataset:
            for i in range(len(dataset)):
                tested_value = dataset[column].iloc[i]
                try:
                    if type(tested_value) == str:
                        tested_value = tested_value.replace(',', '.')
                        a = float(tested_value)
                        dataset.set_value(dataset.index.values[i], column, a)
                except:
                    if column not in columns_to_drop:
                        columns_to_drop.append(column)
                    break
                if tested_value != tested_value or tested_value == float(-999):
                    if column not in dropped_counter.keys():
                        dropped_counter[column] = 1
                    else:
                        dropped_counter[column] += 1

        for column in dropped_counter.keys():
            if dropped_counter[column] >= thresh:
                if column not in columns_to_drop:
                    columns_to_drop.append(column)

        for column in columns_to_drop:
            if column in dataset.keys():
                try:
                    dataset = dataset.drop(column, 1)
                    dropped_columns.append(column)
                except:
                    print('there was a problem droping', column)
                    None
                del columns_to_drop[columns_to_drop.index(column)]

        subjects_to_drop = []
        bad_subjects = {}
        for index, row in dataset.iterrows():
            for element in row:
                if element != element:
                    dataset = dataset.drop(index)
                    break
                elif element == float(-999):
                    if subject_list[index] not in bad_subjects.keys():
                        bad_subjects[subject_list[index]] = [1, index]
                    else:
                        bad_subjects[subject_list[index]][0] += 1
        
        nb_bad_subjects = len(bad_subjects.keys())
        for subject in bad_subjects.keys():
            if bad_subjects[subject][0] >= thresh:
                subjects_to_drop.append(bad_subjects[subject][1])

        for index in subjects_to_drop:
            tested_subject = subject_list[index]
#             try:
#                 dataset = dataset.drop(index)
#                 subject_list = subject_list.drop(index)
#                 dropped_subjects.append(tested_subject)
#             except:
#                 print('there was a problem droping', tested_subject)
    return dataset, dropped_columns, dropped_subjects


def FindMinorClass(label0_index, label1_index):
    # Création d'outils de balancing des classes
    quantity_of_class = {'0': len(label0_index),
                         '1': len(label1_index)}


    nb_minority_class = min(quantity_of_class['0'], quantity_of_class['1'])

    for key in quantity_of_class.keys():
        if nb_minority_class == quantity_of_class[key]:
            minority_class = int(key)

    if minority_class == 0:
        major_class = 1
        minor_class_index = label0_index
        major_class_index = label1_index
    else:
        major_class = 0
        minor_class_index = label1_index
        major_class_index = label0_index

    minor_class_index = np.asarray(minor_class_index)
    return minority_class, major_class, minor_class_index, major_class_index


def CreateRandomBalancedDataset(dataset, minor_class_index, major_class_index, n_repet=1):
    random_major_indexes = []
    for i in range(n_repet):
        my_set = np.random.choice(major_class_index, len(minor_class_index), replace=False)
        random_major_indexes.append(my_set)
    return random_major_indexes


def SelectSubjects(dataset, conditions):
    # sélectionne les sujets qui correpondent à nos conditions
    cond = conditions
    if 4 in cond:
        cond.append(2)
        cond.append(1)
        del cond[cond.index(4)]
    for index, row in dataset.iterrows():
        look_at = row['Type de Conversion']
        if look_at not in cond:
            dataset = dataset.drop(index, 0)
    return dataset


# Definition des paramètres
data_path = path('.').abspath()#chemin d'Acces relatif vers absolu
save_path = data_path / 'results'#doissier au nom de result sera crée pour sauvegarder des trucs
if not save_path.isdir(): # crée le chemin de sauvegarde s'il nexiste pas
    save_path.mkdir()
# df = pd.read_csv(data_path + 'dataset.csv')
df = pd.read_csv(data_path / 'BD_17_11.csv')


rep_number = 100
n_permutations = 1000
subject_list = df['CODE']
print('Tout est OK')

Tout est OK


# Conditions Originales

In [4]:
''' A exécuter en deuxième à chaque fois'''
def CondNames(cond):
    '''permet de convertir les infos (1,2), (0) par exemple en infos intelligibles
    dans notre cas (1,2) correspond à Parkinson, Démence
    et (0) correspond à Disease Free
    On fait donc dans l'exemple Conv vs DF '''
    # ici on convertis les infos de la première colonne (indicée 0)
    if cond[0][0] == cond[0][1]: # si les deux chiffres de la première colonne sont les mêmes
                                 # On vérifie à quelle condition cela correspond
        if cond[0][0] == 1:
            name1 = 'Parkinson Disease'
        elif cond[0][0] == 2:
            name1 = 'Dementia Lewy bodies'
        elif cond[0][0] == 0:
            name1 = 'Disease Free'
    else: # sinon, c'est qu'on a des chiffres différents donc on étudie les "convertis"
        name1 = 'Converted'
        
    # ici on convertis les infos de la deuxième colonne (indicée 1)
    if cond[1] == 0:
        name2 = 'Disease Free'
    elif cond[1] == 2:
        name2 = 'Dementia Lewy bodies'
    if cond[1] == 3:
        name2 = 'Control'
    return name1, name2

def CreateLabels(dataset, cond):
    # génère la liste des étiquettes en fonction du dataset et de la condition
    label1_index = []
    label0_index = []
    for index, row in dataset.iterrows():
        look_at = row['Type de Conversion']
        if look_at == cond[0][0] or look_at == cond[0][1]:
            label0_index.append(index)
        elif look_at == cond[1]:
            label1_index.append(index)
    return label0_index, label1_index

# Ici on définit les conditions : ajouter ou supprimer des conditions
conds_list = [#((2, 2), (3)), #  Démence vs Contrôle
              ((1, 1), (2))] #  Parkinson vs Démence
#               ((1, 1), (3)), #  PD vs COntrôle
#               ((2, 2), (0)), #  Démence vs DF
#               ((1, 1), (0)), #  Parkinson vs DF
#               ((1, 2), (0)), #  Conv vs DF
#               ((1, 2), (3)), #  Conv vs Contrôle
#               ((0, 0), (3))] #  DF vs Contrôle

# columns_to_drop = ['Conversion', 'PDvsDLB', 'DLBvs Ctrl', 'Type de Conversion']
columns_to_drop = ['Age']
MCI = False
print('Tout est OK')

Tout est OK


# Conditions MCI

In [3]:
def ConditionsMCI(o):
    name1, name2 = '',''
    if o == 0:
        name1, name2 = 'RBDMCI', 'RBDnoMCI'
        conditions = [0,1,2]
    elif o == 1:
        name1, name2 = 'RBD', 'Control'
        conditions = [0,1,2,3]
    elif o == 2:
        name1, name2 = 'RBDnoMCI + Control', 'RBDMCI'
        conditions = [0,1,2,3]
    return (name1, name2), conditions

def CreateLabelsMCI(dataset, o):
    label1_index = []
    label0_index = []
    for index, row in dataset.iterrows():
        look_at2 = float(row['MCI au T1'])
        look_at = row['Type de Conversion']
        if o in (0,2):
            if look_at2 == 1:
                label0_index.append(index)
            elif look_at2 == 0:
                label1_index.append(index)
            else:
                dataset = dataset.drop(index, 0)
        elif o == 1:
            if look_at in (0,1,2):
                label0_index.append(index)
            elif look_at == 3:
                label1_index.append(index)
            else:
                dataset = dataset.drop(index, 0)
    return label0_index, label1_index

conds_list = ['trois', 'trucs', 'random']

columns_to_drop = ['ss-type MCI T1 (DxBrain)', 'RBD_MCI single/multiple domain']
MCI = True

# Dataset info for conditions

In [12]:
if __name__ == '__main__':
    # Pour classif Parkinson-Démence :
    # DLB=2 PD=1 Normal=0 Contrôle=3
    for o, cond in enumerate(conds_list):
        dataset = df
        kept_features = []
        if MCI:
            names, conditions = ConditionsMCI(o)
        else:
            names = CondNames(cond)
            conditions = [cond[0][0], cond[0][1], cond[1]]
        # trie les sujets :
        dataset = SelectSubjects(dataset, conditions)s
        # on nettoie plus de sujets pour les conditions MCI
        # nettoie les donnees :
        dataset, dropped_columns, dropped_subjects = CleanDataset(dataset, columns_to_drop)
        # cree les labels :
        if MCI:
            label0_index, label1_index = CreateLabelsMCI(dataset, o)       
            dataset = dataset.drop('MCI au T1', 1)
        else:
            label0_index, label1_index = CreateLabels(dataset, cond)
        dataset = dataset.drop('Type de Conversion', 1)
        for column in dataset:
            kept_features.append(column)

        print('\n%s features were dropped. Kept features :' % len(dropped_columns))
        print(kept_features)
        print(len(dropped_subjects), 'subjects were dropped :')
        print(dropped_subjects)
        # Verifier quelle est la classe minoritaire
        m_class, M_class, m_class_index, M_class_index = FindMinorClass(label0_index, label1_index)
        nb_minority_class = len(m_class_index)
        print('%i %s vs %i %s' % (nb_minority_class, names[m_class], len(M_class_index), names[M_class]))
        print("Il y a %i features et %i sujets." % (dataset.shape[1], len(dataset)))


26 features were dropped. Kept features :
['MCI au T1', 'ss-type MCI T1 (DxBrain)', 'RBD_MCI single/multiple domain', 'Age', 'Educ', 'Sex', 'PriseAnxio_AD', 'PriseAD', 'MMSE', 'Empandir', 'EmpanIndir', 'TrailA', 'TrailB', 'TrailB_TrailA', 'TrailBris', 'ReyTot', 'ReyB', 'ReyRI', 'ReyRD', 'Reyreco', 'FigReyRI', 'FigReyRD', 'FReyco', 'UPSITtotal12', 'UPSITpercexp', 'UPDRS3']
0 subjects were dropped :
[]
16 Dementia Lewy bodies vs 18 Parkinson Disease
Il y a 26 features et 34 sujets.


# Actual classification 

In [17]:
if __name__ == '__main__':
    # Pour classif Parkinson-Démence :
    # DLB=2 PD=1 Normal=0 Contrôle=3
    for o, cond in enumerate(conds_list):
        kept_features = []
        dataset = df
        
        if MCI:
            names, conditions = ConditionsMCI(o)
        else:
            names = CondNames(cond)
            conditions = [cond[0][0], cond[0][1], cond[1]]
#         if 3 in conditions:
#             if 'MCI au T1' not in columns_to_drop:
#                 columns_to_drop.append('MCI au T1')
                
        dataset = SelectSubjects(dataset, conditions)
        dataset, dropped_columns, dropped_subjects = CleanDataset(dataset, columns_to_drop)
        if MCI:
            label0_index, label1_index = CreateLabelsMCI(dataset, o)
            dataset = dataset.drop('MCI au T1', 1)
        else:
            label0_index, label1_index = CreateLabels(dataset, cond)
        dataset = dataset.drop('Type de Conversion', 1)
        for column in dataset:
            kept_features.append(column)
            
        print('\n%s features were dropped. Kept features :' % len(dropped_columns))
        print(kept_features)
        print(len(dropped_subjects), 'subjects were dropped :')
        print(dropped_subjects)

        m_class, M_class, m_class_index, M_class_index = FindMinorClass(label0_index, label1_index)
        nb_minority_class = len(m_class_index)
        
        print('%i %s vs %i %s' % (nb_minority_class, names[m_class], len(M_class_index), names[M_class]))
        if nb_minority_class > 4:
            print("Il y a %i features et %i sujets." % (dataset.shape[1], len(dataset)))
            file_name = '%svs%s.mat' % (names[m_class], names[M_class])
            file_path = save_path / file_name
            if not file_path.isfile():
                labels = [M_class]*nb_minority_class + [m_class]*nb_minority_class
                labels = np.asarray(labels, dtype=int)

                number_of_folds = max(int(nb_minority_class/2),2)  # aura pour équivalent Leave 4 subject Out stratifié
                cv = StratifiedKFold(n_splits=number_of_folds, shuffle=True)
                #clf_choice = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=200, learning_rate=1)
                clf_choice= SVM() #ou mettre RF()  # C'EST ICI QUE JE CHANGE L'ALGO
                
                random_sets = CreateRandomBalancedDataset(dataset, m_class_index, M_class_index, rep_number)

                accuracies = []
                first = True

                len(dataset)
                for perm in range(n_permutations):
                    for my_set in random_sets:
                        data = pd.concat([dataset.loc[my_set], dataset.loc[m_class_index]])
                        data = np.asarray(data)

                        if first == True:
                            labels = [M_class]*nb_minority_class + [m_class]*nb_minority_class
                            labels = np.asarray(labels, dtype=int)
                            first = False

                        clf = clf_choice
                        accuracies.append(cross_val_score(clf, X=data, y=labels, cv=cv, n_jobs=6).mean())

                    labels = permutation(labels)
                donnees = {'data':accuracies}
                savemat(file_path, donnees)
                
            else:
                accuracies = loadmat(file_path)['data'].ravel()
            
            pvalue = 0
            for score in accuracies[rep_number:]:
                if score > np.mean(accuracies[:rep_number]):
                    pvalue+=1/(n_permutations*rep_number)

            print('%0.2f (+/-%0.2f) significatif a p=%0.4f\n' % (np.mean(accuracies[:rep_number]), np.std(accuracies[:rep_number]), pvalue))

        else:
            print('Not enough subjects to perform classification\n')


37 features were dropped. Kept features :
['MCI au T1', 'Age', 'Educ', 'Sex', 'Empandir', 'EmpanIndir', 'Empanpon', 'TrailA', 'TrailB', 'TrailB_TrailA', 'ReyTot', 'ReyB', 'ReyRI', 'ReyRD', 'Reyreco']
0 subjects were dropped :
[]
16 Dementia Lewy bodies vs 32 Control
Il y a 15 features et 48 sujets.
0.88 (+/-0.04) significatif a p=0.0001


38 features were dropped. Kept features :
['MCI au T1', 'Age', 'Educ', 'Sex', 'Empandir', 'EmpanIndir', 'TrailA', 'TrailB', 'TrailB_TrailA', 'ReyTot', 'ReyB', 'ReyRI', 'ReyRD', 'Reyreco']
0 subjects were dropped :
[]
18 Parkinson Disease vs 32 Control
Il y a 14 features et 50 sujets.
0.65 (+/-0.06) significatif a p=0.0731


29 features were dropped. Kept features :
['MCI au T1', 'ss-type MCI T1 (DxBrain)', 'RBD_MCI single/multiple domain', 'Age', 'Educ', 'Sex', 'PriseAnxio_AD', 'PriseAD', 'Empandir', 'EmpanIndir', 'Empanpon', 'TrailA', 'TrailB', 'TrailB_TrailA', 'TrailBris', 'ReyTot', 'ReyB', 'ReyRI', 'ReyRD', 'Reyreco', 'FReyco', 'UPSITtotal12', 'UP

# Classif Stroop Trail

In [ ]:
if __name__ == '__main__':
    # Pour classif Parkinson-Démence :
    # DLB=2 PD=1 Normal=0 Contrôle=3
    o=0
    # colonnes à garder, modifie ce que tu veux
    columns_to_keep = ['Stroop1_temps', 'Stroop1_err', 'Stroop2_temps', 
                       'Stroop2_err', 'Stroop3_temps', 'Stroop3_erreur',
                       'Stroop4_temps', 'Stroop4_err', 'Stroop32temps',
                       'Stroop32err', 'TrailA', 'TrailB', 'TrailB_TrailA',
                       'TrailBris', 'Stroop43temps', 'Stroop43err',
                       'CODE', 'Type de Conversion'] # ne pas toucher cett ligne
    for cond in conds_list:
        kept_features = []
        names = CondNames(cond)
        dataset = df

        for column in dataset:
            if column not in columns_to_keep:
                try:
                    dataset = dataset.drop(column, 1)
                except:
                    print('there was a problem droping', column)
                    
        conditions = [cond[0][0], cond[0][1], cond[1]]
        dataset = SelectSubjects(dataset, conditions)
        dataset, dropped_columns, dropped_subjects = CleanDataset(dataset, columns_to_drop)
        label0_index, label1_index = CreateLabels(dataset, cond)
        dataset = dataset.drop('Type de Conversion', 1)
        
        for column in dataset:
            kept_features.append(column)
            
        print('\n%s features were dropped. Kept features :' % len(dropped_columns))
        print(kept_features)
        print(len(dropped_subjects), 'subjects were dropped :')
        print(dropped_subjects)

        m_class, M_class, m_class_index, M_class_index = FindMinorClass(label0_index, label1_index)
        nb_minority_class = len(m_class_index)
        
        print('%i %s vs %i %s' % (nb_minority_class, names[m_class], len(M_class_index), names[M_class]))
        if nb_minority_class > 4:
            print("Il y a %i features et %i sujets." % (dataset.shape[1], len(dataset)))
            file_name = '%svs%s_stroop_trail.mat' % (names[m_class], names[M_class])
            file_path = save_path / file_name
            if not file_path.isfile():
                labels = [M_class]*nb_minority_class + [m_class]*nb_minority_class
                labels = np.asarray(labels, dtype=int)

                number_of_folds = int(nb_minority_class/2)  # aura pour équivalent Leave 4 subject Out stratifié
                cv = StratifiedKFold(n_splits=number_of_folds, shuffle=True)
                clf_choice = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=200, learning_rate=1)

                random_sets = CreateRandomBalancedDataset(dataset, m_class_index, M_class_index, rep_number)
                print('%i %s vs %i %s' % (len(m_class_index), names[m_class], len(M_class_index), names[M_class]))

                accuracies = []
                first = True

                len(dataset)
                for perm in range(n_permutations+1):
                    for my_set in random_sets:
                        data = pd.concat([dataset.loc[my_set], dataset.loc[m_class_index]])
                        data = np.asarray(data)

                        if first == True:
                            labels = [M_class]*nb_minority_class + [m_class]*nb_minority_class
                            labels = np.asarray(labels, dtype=int)
                            first = False

                        clf = clf_choice
                        accuracies.append(cross_val_score(clf, X=data, y=labels, cv=cv, n_jobs=-1).mean())

                    labels = permutation(labels)
                donnees = {'data':accuracies}
                savemat(file_path, donnees)
                
            else:
                accuracies = loadmat(file_path)['data'].ravel()

            pvalue = 0                
            for score in accuracies[rep_number:]:
                if score > np.mean(accuracies[:rep_number]):
                    pvalue+=1/(n_permutations*rep_number)

            print('%0.2f (+/-%0.2f) significatif a p=%0.4f\n' % (np.mean(accuracies[:rep_number]), np.std(accuracies[:rep_number]), pvalue))

        else:
            print('Not enough subjects to perform classification\n')
        o += 1

# Feature Selection

In [32]:
if __name__ == '__main__':
    # Pour classif Parkinson-Démence :
    # DLB=2 PD=1 Normal=0 Contrôle=3
    rep_number = 100
    for o, cond in enumerate(conds_list):
        kept_features = []
        dataset = df
        
        if MCI:
            names, conditions = ConditionsMCI(o)
        else:
            names = CondNames(cond)
            conditions = [cond[0][0], cond[0][1], cond[1]]
#         if 3 in conditions:
#             if 'MCI au T1' not in columns_to_drop:
#                 columns_to_drop.append('MCI au T1')
                
        dataset = SelectSubjects(dataset, conditions)
        dataset, dropped_columns, dropped_subjects = CleanDataset(dataset, columns_to_drop)
        if MCI:
            label0_index, label1_index = CreateLabelsMCI(dataset, o)
            dataset = dataset.drop('MCI au T1', 1)
        else:
            label0_index, label1_index = CreateLabels(dataset, cond)
        dataset = dataset.drop('Type de Conversion', 1)
        for column in dataset:
            kept_features.append(column)
            
        print('\n%s features were dropped. Kept features :' % len(dropped_columns))
        print(kept_features)
        print(len(dropped_subjects), 'subjects were dropped :')
        print(dropped_subjects)

        m_class, M_class, m_class_index, M_class_index = FindMinorClass(label0_index, label1_index)
        nb_minority_class = len(m_class_index)
        
        print('%i %s vs %i %s' % (nb_minority_class, names[m_class], len(M_class_index), names[M_class]))
        kept_features = np.asarray(kept_features)
        if nb_minority_class > 4:
            print("Il y a %i features et %i sujets." % (dataset.shape[1], len(dataset)))
            labels = [M_class]*nb_minority_class + [m_class]*nb_minority_class
            labels = np.asarray(labels, dtype=int)

            number_of_folds = int(nb_minority_class/2)  # aura pour équivalent Leave 4 subject Out stratifié
            clf_choice = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=200, learning_rate=1)

            random_sets = CreateRandomBalancedDataset(dataset, m_class_index, M_class_index, rep_number)

            accuracies = []
            first = True
            pvalue = 0
            file_name = '%svs%s_features.mat' % (names[m_class], names[M_class])
            file_path = save_path / file_name
            feats = []
            if not file_path.isfile():
                for my_set in random_sets:
                    data = pd.concat([dataset.loc[my_set], dataset.loc[m_class_index]])
                    data = np.asarray(data)

                    if first == True:
                        labels = [M_class]*nb_minority_class + [m_class]*nb_minority_class
                        labels = np.asarray(labels, dtype=int)
                        first = False

                    clf = clf_choice
                    sfs1 = SFS(estimator=clf,
                               k_features=(1,data.shape[1]),
    #                            k_features=3,
                               forward=False,
                               floating=False,
                               scoring='accuracy',
                               verbose=1,
                               cv=5,
                               n_jobs=6)

                    sfs1.fit(data, labels)
                    print('Best combin (ACC= %.3f): %s' % (sfs1.k_score_, kept_features[list(sfs1.k_feature_idx_)].tolist()))
                    feats.append(kept_features[list(sfs1.k_feature_idx_)].tolist())
                sizes = [len(feat) for feat in feats]
                for feat in feats:
                    while len(feat) < max(sizes):
                        feat.append('empty')
                savemat(file_path, {'features': feats, 'score': sfs1.k_score_ })
        else:
            print('Not enough subjects to perform classification\n')


27 features were dropped. Kept features :
['MCI au T1', 'ss-type MCI T1 (DxBrain)', 'RBD_MCI single/multiple domain', 'Age', 'Educ', 'Sex', 'PriseAnxio_AD', 'PriseAD', 'Empandir', 'EmpanIndir', 'Empanpon', 'TrailA', 'TrailB', 'TrailB_TrailA', 'TrailBris', 'ReyTot', 'ReyB', 'ReyRI', 'ReyRD', 'Reyreco', 'FigReyRI', 'FigReyRD', 'FReyco', 'UPSITtotal12', 'UPDRS3']
0 subjects were dropped :
[]
16 Dementia Lewy bodies vs 18 Parkinson Disease
Il y a 25 features et 34 sujets.


Features: 25/25

Best combin (ACC= 0.825): ['RBD_MCI single/multiple domain', 'Age', 'FigReyRI', 'UPDRS3', 'TrailBris', 'ReyTot']


Features: 25/25

Best combin (ACC= 0.842): ['MCI au T1', 'RBD_MCI single/multiple domain', 'Age', 'Sex', 'PriseAD', 'Empandir', 'EmpanIndir', 'Empanpon', 'TrailA', 'ReyTot', 'ReyB', 'FigReyRI']


...

Best combin (ACC= 0.842): ['MCI au T1', 'ss-type MCI T1 (DxBrain)', 'RBD_MCI single/multiple domain', 'Age', 'Sex', 'PriseAD', 'Empandir', 'Empanpon', 'TrailB', 'TrailBris', 'FigReyRD', 'UPSITtotal12']


Features: 25/25

Best combin (ACC= 0.850): ['MCI au T1', 'Age', 'Empandir', 'EmpanIndir', 'Empanpon', 'TrailA', 'ReyRI', 'FReyco', 'UPDRS3']


Features: 1/25Process ForkPoolWorker-3714767:
Process ForkPoolWorker-3714766:
Process ForkPoolWorker-3714768:
Process ForkPoolWorker-3714765:
Process ForkPoolWorker-3714764:
Process ForkPoolWorker-3714763:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tarek/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tarek/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/tarek/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/tarek/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/tarek/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/tarek/anaconda3/lib/py

TypeError: catching classes that do not inherit from BaseException is not allowed

In [29]:
import operator
for file in [f for f in save_path.files() if f.endswith('_features.mat')]:
    a = loadmat(file)
    features = a['features'].ravel()
    for i, feature in enumerate(features):
        features[i] = feature.replace(" ", "")
    scores = a['score']
    del a
    compte = {}.fromkeys(set(features),0)
    for feature in features:
        compte[feature] += 1
    del compte['empty']
    best_features = sorted(compte.items(), key=operator.itemgetter(1), reverse=True)
    print(file.name[:-13].replace('vs', ' vs ') + ':', best_features, '\n')


Dementia Lewy bodies vs Control: [('MCIauT1', 100), ('Empandir', 49), ('Sex', 19), ('ReyRD', 16), ('Reyreco', 8), ('EmpanIndir', 7), ('Educ', 3), ('TrailB', 3), ('ReyTot', 2), ('ReyB', 2), ('TrailA', 2), ('Age', 1)] 

Dementia Lewy bodies vs Disease Free: [('UPSITtotal12', 71), ('PriseAnxio_AD', 64), ('MCIauT1', 45), ('Sex', 37), ('TrailBris', 24), ('RBD_MCIsingle/multipledomain', 22), ('Age', 21), ('Empandir', 20), ('PriseAD', 20), ('Reyreco', 18), ('TrailB', 17), ('EmpanIndir', 14), ('TrailA', 13), ('TrailB_TrailA', 11), ('ss-typeMCIT1(DxBrain)', 11), ('UPDRS3', 11), ('Educ', 11), ('ReyRI', 10), ('FReyco', 8), ('ReyRD', 7), ('Empanpon', 7), ('ReyB', 6), ('ReyTot', 5)] 

Parkinson Disease vs Disease Free: [('Sex', 48), ('Empanpon', 48), ('PriseAD', 46), ('TrailBris', 44), ('ReyRI', 42), ('MCIauT1', 38), ('RBD_MCIsingle/multipledomain', 33), ('UPDRS3', 32), ('ReyB', 30), ('PriseAnxio_AD', 29), ('ss-typeMCIT1(DxBrain)', 28), ('Reyreco', 28), ('Age', 28), ('ReyRD', 27), ('EmpanIndir', 26

# Classif with selected features

In [ ]:
if __name__ == '__main__':
    # Pour classif Parkinson-Démence :
    # DLB=2 PD=1 Normal=0 Contrôle=3
    o=0
#     columns_to_keep = [['TrailBris'] + ['Empanpon', 'MCI au T1', 'UPDRS3'],
#                        ['Educ', 'Stroop43err','TrailBris'],
#                        ['MCI au T1'],
#                        ['TrailBris'],
#                        ['ReyTot', 'ReyRI', 'Age'] + ['ReyB', 'EmpanIndir'],
#                        ['Educ', 'Sex']]

#     columns_to_keep = [['TrailBris', 'Empanpon' ]] # pour changer les features

    rep_number = 5
    n_permutations = 1000
    columns_to_keep = [['TrailBris', 'ReyTot']] # pour changer les features
    conds_list = [conds_list[0]] # pour selectionner la condition dans cond_list
    
    for cond in conds_list:
        kept_features = []
        names = CondNames(cond)
        dataset = df
        
        columns_to_keep[o] += ['CODE', 'Type de Conversion']
        for column in dataset:
            if column not in columns_to_keep[o]:
                try:
                    dataset = dataset.drop(column, 1)
                except:
                    print('there was a problem droping', column)
                    
        conditions = [cond[0][0], cond[0][1], cond[1]]
        dataset = SelectSubjects(dataset, conditions)
        dataset, dropped_columns, dropped_subjects = CleanDataset(dataset, columns_to_drop)
        label0_index, label1_index = CreateLabels(dataset, cond)
        dataset = dataset.drop('Type de Conversion', 1)
    
        for column in dataset:
            kept_features.append(column)
            
        print('\n%s features were dropped. Kept features :' % len(dropped_columns))
        print(kept_features)
        print(len(dropped_subjects), 'subjects were dropped :')
        print(dropped_subjects)

        m_class, M_class, m_class_index, M_class_index = FindMinorClass(label0_index, label1_index)
        nb_minority_class = len(m_class_index)
        
        print('%i %s vs %i %s' % (nb_minority_class, names[m_class], len(M_class_index), names[M_class]))
        if nb_minority_class > 4:
            print("Il y a %i features et %i sujets." % (dataset.shape[1], len(dataset)))
            file_name = '%svs%s_with_selfeatures.mat' % (names[m_class], names[M_class])
            file_path = save_path / file_name
            if not file_path.isfile():
                labels = [M_class]*nb_minority_class + [m_class]*nb_minority_class
                labels = np.asarray(labels, dtype=int)

                number_of_folds = int(nb_minority_class/2)  # aura pour équivalent Leave 4 subject Out stratifié
                cv = StratifiedKFold(n_splits=number_of_folds, shuffle=True)
                clf_choice = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=200, learning_rate=1)

                random_sets = CreateRandomBalancedDataset(dataset, m_class_index, M_class_index, rep_number)

                accuracies = []
                first = True
                pvalue = 0

                len(dataset)
                for perm in range(n_permutations+1):
                    for my_set in random_sets:
                        data = pd.concat([dataset.loc[my_set], dataset.loc[m_class_index]])
                        data = np.asarray(data)

                        if first == True:
                            labels = [M_class]*nb_minority_class + [m_class]*nb_minority_class
                            labels = np.asarray(labels, dtype=int)
                            first = False

                        clf = clf_choice
                        accuracies.append(cross_val_score(clf, X=data, y=labels, cv=cv, n_jobs=-1).mean())

                    labels = permutation(labels)
                donnees = {'data':accuracies}
                savemat(file_path, donnees)
                
            else:
                accuracies = loadmat(file_path)['data'].ravel()
            
            pvalue = 0                
            for score in accuracies[rep_number:]:
                if score > np.mean(accuracies[:rep_number]):
                    pvalue+=1/(n_permutations*rep_number)

            print('%0.2f (+/-%0.2f) significatif a p=%0.4f\n' % (np.mean(accuracies[:rep_number]), np.std(accuracies[:rep_number]), pvalue))

        else:
            print('Not enough subjects to perform classification\n')
        o += 1

# Each feature one by one

In [8]:
if __name__ == '__main__':
    # Pour classif Parkinson-Démence :
    # DLB=2 PD=1 Normal=0 Contrôle=3
#     columns_to_keep = [['TrailBris'] + ['Empanpon', 'MCI au T1', 'UPDRS3'],
#                        ['Educ', 'Stroop43err','TrailBris'],
#                        ['MCI au T1'],
#                        ['TrailBris'],
#                        ['ReyTot', 'ReyRI', 'Age'] + ['ReyB', 'EmpanIndir'],
#                        ['Educ', 'Sex']]

#     columns_to_keep = [['TrailBris', 'Empanpon' ]] # pour changer les features

    rep_number = 5
    n_permutations = 1000
    
    feature_list = ['MCI au T1', 'ss-type MCI T1 (DxBrain)',
       'RBD_MCI single/multiple domain', 'Apnée', 'Age', 'Educ', 'Sex',
       'PriseAnxio_AD', 'PriseAD', 'BDI', 'BAI', 'Epworth', 'MOCA', 'MMSE',
       'Empandir', 'EmpanIndir', 'Empanpon', 'Stroop1_temps', 'Stroop1_err',
       'Stroop2_temps', 'Stroop2_err', 'Stroop3_temps', 'Stroop3_erreur',
       'Stroop4_temps', 'Stroop4_err', 'Stroop32temps', 'Stroop32err',
       'TrailA', 'TrailB', 'TrailB_TrailA', 'TrailBris', 'Stroop43temps',
       'Stroop43err', 'Flusem', 'Flupho', 'ReyTot', 'ReyB', 'ReyRI', 'ReyRD',
       'Reyreco', 'FigReyRI', 'FigReyRD', 'FReyco', 'Blocspon', 'Clom',
       'SymptoRBD', 'UPSITtotal12', 'UPSITtotal40', 'UPSITpercexp',
       'FM100total', 'FM100percnormal', 'UPDRS3']
    
    conds_list = [conds_list[0]] # pour selectionner la condition dans cond_list
    
    for feature in feature_list:
        kept_features = []
        names = CondNames(cond)
        dataset = df

        columns_to_keep = ['CODE', 'Type de Conversion']
        columns_to_keep.append(feature)
        for column in dataset:
            if column not in columns_to_keep:
                try:
                    dataset = dataset.drop(column, 1)
                except:
                    print('there was a problem droping', column)

        conditions = [cond[0][0], cond[0][1], cond[1]]
        dataset = SelectSubjects(dataset, conditions)
        dataset, dropped_columns, dropped_subjects = CleanDataset(dataset, columns_to_drop)
        label0_index, label1_index = CreateLabels(dataset, cond)
        dataset = dataset.drop('Type de Conversion', 1)

        for column in dataset:
            kept_features.append(column)

        print('\n%s features were dropped. Kept features :' % len(dropped_columns))
        print(kept_features)
        print(len(dropped_subjects), 'subjects were dropped :')
        print(dropped_subjects)

        m_class, M_class, m_class_index, M_class_index = FindMinorClass(label0_index, label1_index)
        nb_minority_class = len(m_class_index)

        print('%i %s vs %i %s' % (nb_minority_class, names[m_class], len(M_class_index), names[M_class]))
        if nb_minority_class > 4:
            print("Il y a %i features et %i sujets." % (dataset.shape[1], len(dataset)))
            file_name = '%svs%s_with_selfeatures.mat' % (names[m_class], names[M_class])
            file_path = save_path / file_name
            if not file_path.isfile():
                labels = [M_class]*nb_minority_class + [m_class]*nb_minority_class
                labels = np.asarray(labels, dtype=int)

                number_of_folds = int(nb_minority_class/2)  # aura pour équivalent Leave 4 subject Out stratifié
                cv = StratifiedKFold(n_splits=number_of_folds, shuffle=True)
                clf_choice = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=200, learning_rate=1)

                random_sets = CreateRandomBalancedDataset(dataset, m_class_index, M_class_index, rep_number)

                accuracies = []
                first = True
                pvalue = 0

                len(dataset)
                for perm in range(n_permutations+1):
                    for my_set in random_sets:
                        data = pd.concat([dataset.loc[my_set], dataset.loc[m_class_index]])
                        data = np.asarray(data)

                        if first == True:
                            labels = [M_class]*nb_minority_class + [m_class]*nb_minority_class
                            labels = np.asarray(labels, dtype=int)
                            first = False

                        clf = clf_choice
                        accuracies.append(cross_val_score(clf, X=data, y=labels, cv=cv, n_jobs=-1).mean())

                    labels = permutation(labels)
                donnees = {'data':accuracies}
                savemat(file_path, donnees)

            else:
                accuracies = loadmat(file_path)['data'].ravel()

            pvalue = 0                
            for score in accuracies[rep_number:]:
                if score > np.mean(accuracies[:rep_number]):
                    pvalue+=1/(n_permutations*rep_number)

            print('%0.2f (+/-%0.2f) significatif a p=%0.4f\n' % (np.mean(accuracies[:rep_number]), np.std(accuracies[:rep_number]), pvalue))

        else:
            print('Not enough subjects to perform classification\n')
        o += 1

NameError: name 'columns_to_keep' is not defined

# Single Feature

In [5]:
if __name__ == '__main__':
    # Pour classif Parkinson-Démence :
    # DLB=2 PD=1 Normal=0 Contrôle=3
    cond = conds_list[0]
    kept_features = []
    names = CondNames(cond)
    dataset = df
    
    conditions = [cond[0][0], cond[0][1], cond[1]]
    dataset = SelectSubjects(dataset, conditions)
    dataset, dropped_columns, dropped_subjects = CleanDataset(dataset, columns_to_drop)
    label0_index, label1_index = CreateLabels(dataset, cond)
    dataset = dataset.drop('Type de Conversion', 1)
    
    for column in dataset:
        kept_features.append(column)

    print('\n%s features were dropped. Kept features :' % len(dropped_columns))
    print(kept_features)
    print(len(dropped_subjects), 'subjects were dropped :')
    print(dropped_subjects)

    m_class, M_class, m_class_index, M_class_index = FindMinorClass(label0_index, label1_index)
    nb_minority_class = len(m_class_index)

    print('%i %s vs %i %s' % (nb_minority_class, names[m_class], len(M_class_index), names[M_class]))
    if nb_minority_class > 4:
        print("Il y a %i features et %i sujets." % (dataset.shape[1], len(dataset)))
        file_name = '%svs%s_single_feature.mat' % (names[m_class], names[M_class])
        file_path = save_path / file_name
        if not file_path.isfile():
            labels = [M_class]*nb_minority_class + [m_class]*nb_minority_class
            labels = np.asarray(labels, dtype=int)

            number_of_folds = int(nb_minority_class/2)  # aura pour équivalent Leave 4 subject Out stratifié
            cv = StratifiedKFold(n_splits=number_of_folds, shuffle=True)
            clf_choice = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=200, learning_rate=1)

            random_sets = CreateRandomBalancedDataset(dataset, m_class_index, M_class_index, rep_number)

            accuracies = []
            first = True
            accuracies = np.zeros((len(kept_features), n_permutations, len(random_sets)))
            for perm in range(n_permutations):
                for i in range(len(random_sets)):
                    my_set = random_sets[i]
                    data = pd.concat([dataset.loc[my_set], dataset.loc[m_class_index]])
                    data = np.asarray(data)

                    if first == True:
                        labels = [M_class]*nb_minority_class + [m_class]*nb_minority_class
                        labels = np.asarray(labels, dtype=int)
                        first = False

                    for k in range(data.shape[1]):
                        clf = clf_choice
                        accuracies[k, perm, i] = cross_val_score(clf, X=data[:,k].reshape(-1, 1), y=labels, cv=cv, n_jobs=-1).mean()

                labels = permutation(labels)
                
            donnees = {'data':accuracies}
            savemat(file_path, donnees)
        else:
            accuracies = loadmat(file_path)['data']
            
        for i, feature in enumerate(kept_features):
            pvalue = 0
            for perm in range(1, accuracies.shape[1]):
                for score in accuracies[i, perm, :]:
                    if score > accuracies[i, 0, :].mean():
                        pvalue+=1/5000
            print('%s : %0.2f (+/-%0.2f) significatif a p=%0.4f' % (feature, accuracies[i, 0, :].mean(), accuracies[i, 0, :].std(), pvalue))

    else:
        print('Not enough subjects to perform classification\n')


29 features were dropped. Kept features :
['Conversion', 'PDvsDLB', 'MCI au T1', 'ss-type MCI T1 (DxBrain)', 'Apnée', 'Educ', 'Sex', 'PriseAnxio_AD', 'PriseAD', 'MMSE', 'Empandir', 'EmpanIndir', 'TrailA', 'TrailB', 'TrailB_TrailA', 'TrailBris', 'ReyTot', 'ReyB', 'ReyRI', 'ReyRD', 'Reyreco', 'FigReyRI', 'FigReyRD', 'FReyco', 'UPSITtotal12', 'UPSITpercexp', 'UPDRS3']
0 subjects were dropped :
[]
16 Dementia Lewy bodies vs 18 Parkinson Disease
Il y a 27 features et 34 sujets.


KeyboardInterrupt: 